In [1]:
using Requests
using LibExpat
#using Bio

In [2]:
# single letter amino acid codes and STOP
ipuac="ACDEFGHIKLMNPQRSTVWY*";

Make a list of all pairs of IPUAC amino acid codes including self pairings
which represent the replacement of the first aa with the second aa.

In [3]:
# all transitions from one aa to another (inc symmeteric)
allalt = []
for aa in ipuac
    for bb in ipuac
        # if(aa != bb)  # now including silent subsitutions   
            push!(allalt, string(aa) * string(bb))
        #end
    end
end
length(allalt)

441

Allowing silent subsitutions here means we may have to check for exactly equal codons later  

```human_aa_codon_freq.txt```
is derived from the first three columns of a table I think I got from wikiopedia   
a year or so ago when I was thinking about gray code cubes for protien sequences.  
I am not having luck finding the exact source just now, 
but here is a similar table from a different source  
http://www.genscript.com/tools/codon-frequency-table  
Here here just breaking file down into maps for  
 - amino acid to codon list 
 - codon to bias
 - codon to amino acid

In [4]:
aa_codon   = Dict{String, Array{String,1}}() 
codon_bias = Dict{String, Float64}()
codon_aa   = Dict{String, String}()

for line in readlines("./human_aa_codon_freq.txt")
    (aa, codon, freq) = split(line)
    if haskey(aa_codon, aa)
        push!(aa_codon[aa], codon)
    else
        aa_codon[aa] = [codon]
    end
    codon_bias[codon] =  float(freq)
    codon_aa[codon] = aa
end   

Although we have generated data for all possible replacements of one
amino acid for another, PR has indicated we will be limiting ourselves to
replacments due to Single Nuclotide Variations (SNV)s which eliminates
a fair number of choices.   
It also means we could use a function to check if   
a pair of codons differ by one nt.

In [5]:
# check that a reference->variant pair of codons differ only by a single nucleotide
function is_snv(r::String, v::String)
    n = 0
    for i in 1:3
        n+=r[i]==v[i]?0:1
    end
    return n==1
end

is_snv (generic function with 1 method)

Given an amino acid replacement we want the codon choices for both reference and variant.  
since each there can be multiple choices each with a bias,  
we can package them up neatly as a `Composite type` which can then be named and passed around as a unit.  

In [6]:
# By defining a Composite Type
# we can get variable names in in the json
type Replacement
    ref_codon::String
    var_codon::String
    likelyhood::Float64
end

Use the composite type we just defined by creating a dictionary  
from an IPUAC amino acid pair replacement string  
to an array of our __Replacement__s

In [7]:
lookup = Dict{String, Array{Replacement,1}}()

scale=10000 #  to truncate decimals 
for alteration in allalt  
    reference = string(alteration[1]) 
    variant = string(alteration[2])
    #alteration = Key(alteration[1:2])
    for ref_codon in aa_codon[reference]
        #println(ref_codon)
        for var_codon in aa_codon[variant]      
            if is_snv(ref_codon, var_codon)
                likelyhood = (round(codon_bias[ref_codon] * codon_bias[var_codon] * scale)/scale)
                if haskey(lookup, alteration)                     
                    push!(lookup[alteration], Replacement(ref_codon, var_codon, likelyhood))
                else 
                   lookup[alteration] = [Replacement(ref_codon, var_codon, likelyhood)]
                end
            end
        end
    end
    if haskey(lookup, alteration)
        sort!(lookup[alteration], by = x -> x.likelyhood, rev = true)
    end
end

In [8]:
# to determine the type of the value a variable is pointing at 
typeof(lookup)

Dict{String,Array{Replacement,1}}

In [9]:
# the type system is complete and coherent (always a DAG)
supertype(typeof(lookup))

Associative{String,Array{Replacement,1}}

Dump the lookup dictionary _including our custom type_ as json for future use.

In [10]:
fh = open("replacement_codon_score.json", "w")
write(fh, JSON.json(lookup))
close(fh)

In practice variant nomenclature is not writ in stone yet  
but a a good start is [HGVS](http://varnomen.hgvs.org/recommendations/general/)  

basicly:  
   - a sequence-identifier and colon 
   - a sequence type code and dot
   - a numeric offset
   - what changed to what  

e.g. `NM_000000.0:c.12345611G>A`


To get that last bit, we can start with a function that reports 
what changed to what and where within a pair of codons.
then given a codon's offset into a sequence we can know the SNV's offset.


In [11]:
# define a new composite type to house 
# the transition and location (within the codon)
type Varloc 
    index::Integer
    transition::String
end

# report what varied where within a codon
function SNV(r::Replacement)
    a=r.ref_codon;b=r.var_codon
    for i in 1:3 
        if a[i]!=b[i] 
            return Varloc(i-1, string(a[i]) * ">" * string(b[i])) 
        end
    end
end

SNV (generic function with 1 method)

[Oncokb API](http://oncokb.org/#/dataAccess) is where we find the variants to try working with.
[citation](DOI: 10.1200/PO.17.00011)

In [12]:
# read in the Onckb variant file
oncodb = JSON.parse(readstring("variants.json"));

In [13]:
#n = 8752;
# pick a random record to look at
n = Int(floor(length(oncodb)*rand()))
println(n)

4060


In [14]:
display(oncodb[n])    

Dict{String,Any} with 9 entries:
  "uniqueId"        => "ERBB3&R670Q&R670Q&MUTATION&missense_variant&670&670&R&Q"
  "name"            => "R670Q"
  "gene"            => Dict{String,Any}(Pair{String,Any}("curatedIsoform","ENST…
  "consequence"     => Dict{String,Any}(Pair{String,Any}("term","missense_varia…
  "variantResidues" => "Q"
  "alteration"      => "R670Q"
  "refResidues"     => "R"
  "proteinStart"    => 670
  "proteinEnd"      => 670

In [15]:
oncodb[n]["gene"]

Dict{String,Any} with 8 entries:
  "curatedIsoform" => "ENST00000267101"
  "name"           => "erb-b2 receptor tyrosine kinase 3"
  "curatedRefSeq"  => "NM_001982.3"
  "tsg"            => false
  "entrezGeneId"   => 2065
  "oncogene"       => true
  "geneAliases"    => Any["ErbB-3","p180-ErbB3","p45-sErbB3","LCCS2","c-erbB-3"…
  "hugoSymbol"     => "ERBB3"

In [16]:
k = oncodb[n]["refResidues"] * oncodb[n]["variantResidues"]
variant_choices = lookup[k]
# length(variant_choices)
display(variant_choices)

2-element Array{Replacement,1}:
 Replacement("CGG","CAG",0.146) 
 Replacement("CGA","CAA",0.0297)

The RefSeq should give us a variant substrate sequence 

In [17]:
function fetch_sequence(refseq_id::String)
    # refseq_id = oncodb[n]["gene"]["curatedRefSeq"]
    efetch = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nuccore&rettype=fasta&retmode=text&id=" 
    resp = get(efetch * refseq_id)
    
    if resp.status != 200  
        println("Didnt get sequence for %s", refseq_id) 
    end   
    page = readstring(resp)
    lines = split(page, "\n")
    # loose the defline  > id ,,,
    lines[1] = ""
    return join(lines)
end        

fetch_sequence (generic function with 1 method)

In [18]:
# n = Int(floor(length(oncodb)*rand()))
# println(n)


refseq_id = oncodb[n]["gene"]["curatedRefSeq"]
seq = fetch_sequence(refseq_id)
offset = 3 * oncodb[n]["proteinStart"]
refseq_codon = seq[offset:offset + 2]

println("Codon found is " * refseq_codon * " which encodes Amino acid " * codon_aa[refseq_codon])
println("Reference residue reported is " * oncodb[n]["refResidues"])

for i in 0:3
    foo = offset + i
    bar = foo + 2
    cdn = seq[foo:bar]
    print(i) 
    print("\t" *cdn * "\t")
    println(codon_aa[cdn] ) 
end

Codon found is TTT which encodes Amino acid F
Reference residue reported is R
0	TTT	F
1	TTT	F
2	TTC	F
3	TCG	S


This makes me sad.  
Given the offset and a curated isoform   
the declared reference residue does not dependably appear in   
any nearby frame shifted window. 

http://www.ensembl.org/Homo_sapiens/DataExport/Protein?component=ProteinSeq;data_type=Transcript;db=core;g=ENSG00000157764;r=7:140719327-140924764;t=ENST00000288602


http://www.ensembl.org/Homo_sapiens/DataExport/Protein?component=ProteinSeq;data_type=Transcript;db=core;t=ENST00000288602


try Eutils

"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=nucleotide&term=" * refseq_id

curl -s  "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=nucleotide&term=NM_020529.2" | xmlstarlet format
<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE eSearchResult PUBLIC "-//NLM//DTD esearch 20060628//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20060628/esearch.dtd">
<eSearchResult>
  <Count>1</Count>
  <RetMax>1</RetMax>
  <RetStart>0</RetStart>
  <IdList>
    <Id>168693660</Id>
  </IdList>
  <TranslationSet/>
  <QueryTranslation/>
</eSearchResult>

"./eSearchResult/IdList/Id"

curl -s "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=nucleotide&db=protein&id=168693660" | xmlstarlet format

<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE eLinkResult PUBLIC "-//NLM//DTD elink 20101123//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20101123/elink.dtd">
<eLinkResult>
  <LinkSet>
    <DbFrom>nuccore</DbFrom>
    <IdList>
      <Id>168693660</Id>
    </IdList>
    <LinkSetDb>
      <DbTo>protein</DbTo>
      <LinkName>nuccore_protein</LinkName>
      <Link>
        <Id>10092619</Id>
      </Link>
    </LinkSetDb>
  </LinkSet>
</eLinkResult>

"./eLinkResult/LinkSet/LinkSetDb/Link/Id"



curl -s "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=protein&rettype=fasta&id=10092619"

>NP_065390.1 NF-kappa-B inhibitor alpha [Homo sapiens]
MFQAAERPQEWAMEGPRDGLKKERLLDDRHDSGLDSMKDEEYEQMVKELQEIRLEPQEVPRGSEPWKQQL
TEDGDSFLHLAIIHEEKALTMEVIRQVKGDLAFLNFQNNLQQTPLHLAVITNQPEIAEALLGAGCDPELR
DFRGNTPLHLACEQGCLASVGVLTQSCTTPHLHSILKATNYNGHTCLHLASIHGYLGIVELLVSLGADVN
AQEPCNGRTALHLAVDLQNPDLVSLLLKCGADVNRVTYQGYSPYQLTWGRPSTRIQQQLGQLTLENLQML
PESEDEESYDTESEFTEFTEDELPYDDCVFGGQRLTL


"alteration"      => "S36Y"

MFQAAERPQEWAMEGPRDGLKKERLLDDRHDSGLD S MKDEEYEQMVKELQEIRLEPQEVPRGSEPWKQQL

that works

In [19]:
# there is probably a lib to do this much better somewhere 
function strip_tags(element::String)
    start = search(element, ">") + 1
    stop =  rsearch(element, "<") - 1
    return element[first(start):first(stop)]
end    

strip_tags (generic function with 1 method)

In [20]:
strip_tags("<foo>bar</foo>")

"bar"

In [21]:

function cdna_refseq_to_protein(refseq_id::String)
    
    eutils = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils"
    esrch  = eutils * "/esearch.fcgi?db=nucleotide&term="
    elink  = eutils * "/elink.fcgi?dbfrom=nucleotide&db=protein&id="
    efetch = eutils * "/efetch.fcgi?db=protein&rettype=fasta&id="
    
    srch_xpath = xpath"/eSearchResult/IdList/Id"
    link_xpath = xpath"/eLinkResult/LinkSet/LinkSetDb/Link/Id"
    
    # print("xpath type is  ")  
    # println(typeof(srch_xpath))
    
    srch_resp = get(esrch * refseq_id)
    if srch_resp.status != 200  
        println("Didnt get search result for " * refseq_id) 
    end
    srch_el_tree = xp_parse(readstring(srch_resp))
    
    # dump(srch_el_tree)
    # alt syntax
    # el = xpath(srch_el_tree, srch_xpath)
    
    ckrit_id = strip_tags(string(srch_el_tree[srch_xpath][1]))  
    display(ckrit_id)
    
    link_resp = get(elink * ckrit_id) 
    if link_resp.status != 200  
        println("Didnt get link result for " * refseq_id *  " using " * ckrit_id) 
    end
    link_el_tree = xp_parse(readstring(link_resp))
    
    display(link_el_tree[link_xpath])
    prot_id = strip_tags(string(link_el_tree[link_xpath][1]))
    display(prot_id)
    
    fetch_resp = get(efetch * prot_id)
    if fetch_resp.status != 200  
        println("Didnt fetch fasta for " * refseq_id  * " using " * ckrit_id * " with " * prot_id) 
    end
    return fasta = readstring(fetch_resp)
end

    

cdna_refseq_to_protein (generic function with 1 method)

In [22]:
println(refseq_id)
prot_fasta = cdna_refseq_to_protein(refseq_id)
println(prot_fasta)

NM_001982.3


"317171925"

1-element Array{LibExpat.ETree,1}:
 <Id>54792100</Id>

"54792100"

>NP_001973.2 receptor tyrosine-protein kinase erbB-3 isoform 1 precursor [Homo sapiens]
MRANDALQVLGLLFSLARGSEVGNSQAVCPGTLNGLSVTGDAENQYQTLYKLYERCEVVMGNLEIVLTGH
NADLSFLQWIREVTGYVLVAMNEFSTLPLPNLRVVRGTQVYDGKFAIFVMLNYNTNSSHALRQLRLTQLT
EILSGGVYIEKNDKLCHMDTIDWRDIVRDRDAEIVVKDNGRSCPPCHEVCKGRCWGPGSEDCQTLTKTIC
APQCNGHCFGPNPNQCCHDECAGGCSGPQDTDCFACRHFNDSGACVPRCPQPLVYNKLTFQLEPNPHTKY
QYGGVCVASCPHNFVVDQTSCVRACPPDKMEVDKNGLKMCEPCGGLCPKACEGTGSGSRFQTVDSSNIDG
FVNCTKILGNLDFLITGLNGDPWHKIPALDPEKLNVFRTVREITGYLNIQSWPPHMHNFSVFSNLTTIGG
RSLYNRGFSLLIMKNLNVTSLGFRSLKEISAGRIYISANRQLCYHHSLNWTKVLRGPTEERLDIKHNRPR
RDCVAEGKVCDPLCSSGGCWGPGPGQCLSCRNYSRGGVCVTHCNFLNGEPREFAHEAECFSCHPECQPME
GTATCNGSGSDTCAQCAHFRDGPHCVSSCPHGVLGAKGPIYKYPDVQNECRPCHENCTQGCKGPELQDCL
GQTLVLIGKTHLTMALTVIAGLVVIFMMLGGTFLYWRGRRIQNKRAMRRYLERGESIEPLDPSEKANKVL
ARIFKETELRKLKVLGSGVFGTVHKGVWIPEGESIKIPVCIKVIEDKSGRQSFQAVTDHMLAIGSLDHAH
IVRLLGLCPGSSLQLVTQYLPLGSLLDHVRQHRGALGPQLLLNWGVQIAKGMYYLEEHGMVHRNLAARNV
LLKSPSQVQVADFGVADLLPPDDKQLLYSEAKTPIKWMALESIHFGKYTHQSDVWSYGVT

In [23]:
tmp = split(prot_fasta, "\n")
tmp[1] = ""
prot_seq = join(tmp)
println(prot_seq)

MRANDALQVLGLLFSLARGSEVGNSQAVCPGTLNGLSVTGDAENQYQTLYKLYERCEVVMGNLEIVLTGHNADLSFLQWIREVTGYVLVAMNEFSTLPLPNLRVVRGTQVYDGKFAIFVMLNYNTNSSHALRQLRLTQLTEILSGGVYIEKNDKLCHMDTIDWRDIVRDRDAEIVVKDNGRSCPPCHEVCKGRCWGPGSEDCQTLTKTICAPQCNGHCFGPNPNQCCHDECAGGCSGPQDTDCFACRHFNDSGACVPRCPQPLVYNKLTFQLEPNPHTKYQYGGVCVASCPHNFVVDQTSCVRACPPDKMEVDKNGLKMCEPCGGLCPKACEGTGSGSRFQTVDSSNIDGFVNCTKILGNLDFLITGLNGDPWHKIPALDPEKLNVFRTVREITGYLNIQSWPPHMHNFSVFSNLTTIGGRSLYNRGFSLLIMKNLNVTSLGFRSLKEISAGRIYISANRQLCYHHSLNWTKVLRGPTEERLDIKHNRPRRDCVAEGKVCDPLCSSGGCWGPGPGQCLSCRNYSRGGVCVTHCNFLNGEPREFAHEAECFSCHPECQPMEGTATCNGSGSDTCAQCAHFRDGPHCVSSCPHGVLGAKGPIYKYPDVQNECRPCHENCTQGCKGPELQDCLGQTLVLIGKTHLTMALTVIAGLVVIFMMLGGTFLYWRGRRIQNKRAMRRYLERGESIEPLDPSEKANKVLARIFKETELRKLKVLGSGVFGTVHKGVWIPEGESIKIPVCIKVIEDKSGRQSFQAVTDHMLAIGSLDHAHIVRLLGLCPGSSLQLVTQYLPLGSLLDHVRQHRGALGPQLLLNWGVQIAKGMYYLEEHGMVHRNLAARNVLLKSPSQVQVADFGVADLLPPDDKQLLYSEAKTPIKWMALESIHFGKYTHQSDVWSYGVTVWELMTFGAEPYAGLRLAEVPDLLEKGERLAQPQICTIDVYMVMVKCWMIDENIRPTFKELANEFTRMARDPPRYLVIKRESGPGIAPGPEPHGLTNKKL

In [24]:
typeof(seq[5])

Char

In [25]:
# do not expect sense to matter for cDNA 
function revcomp(seq::String)
    complement = Dict{Char,Char}()
    complement['G'] = 'C' 
    complement['T'] = 'A' 
    complement['A'] = 'T' 
    complement['C'] = 'G'
    return reverse(replace(seq, ".", (x -> complement[x])))
end    

revcomp (generic function with 1 method)

In [26]:
# revcomp(seq)

In [27]:
# seq

In [28]:
function translate(seq::String)
    join([codon_aa[seq[c:(c+2)]] for c in 1:3:length(seq)-2])
end

translate (generic function with 1 method)

In [29]:
# vanity unit test
translate("ACGGAATGC")

"TEC"

In [30]:
frame1 = translate(seq)
r1 = search(frame1, prot_seq)

93:1434

In [31]:
frame2 = translate(seq[2:end])
r2 = search(frame2, prot_seq)

0:-1

In [32]:
frame3 = translate(seq[3:end])
r3 = search(frame3, prot_seq)

0:-1

In [33]:
frame4 = translate(revcomp(seq))
r4 = search(frame4, prot_seq)

0:-1

In [34]:
frame5 = translate(revcomp(seq[2:end]))
r5 = search(frame5, prot_seq)

0:-1

In [35]:
frame6 = translate(revcomp(seq[3:end]))
r6 = search(frame6, prot_seq)

0:-1

In [36]:
println("Reported amino: " * oncodb[n]["refResidues"])

frame = [r1,r2,r3]
for  j in 1:3
    if frame[j].stop > 0
        for i in -6:6
             offset = 3 * (oncodb[n]["proteinStart"] + frame[j].start -1) + j - 1 + i 
             refseq_codon = seq[offset:offset + 2]
            println("Found amino @: " * string(i) * "\t" * codon_aa[refseq_codon])
        end
    end        
end
#println(refseq_codon * " at " * offset)

Reported amino: R
Found amino @: -6	A
Found amino @: -5	R
Found amino @: -4	A
Found amino @: -3	P
Found amino @: -2	R
Found amino @: -1	G
Found amino @: 0	D
Found amino @: 1	I
Found amino @: 2	F
Found amino @: 3	S
Found amino @: 4	Q
Found amino @: 5	R
Found amino @: 6	E


In [48]:
# 


println(frame1[93:1434])
println("")
println(prot_seq)

MRANDALQVLGLLFSLARGSEVGNSQAVCPGTLNGLSVTGDAENQYQTLYKLYERCEVVMGNLEIVLTGHNADLSFLQWIREVTGYVLVAMNEFSTLPLPNLRVVRGTQVYDGKFAIFVMLNYNTNSSHALRQLRLTQLTEILSGGVYIEKNDKLCHMDTIDWRDIVRDRDAEIVVKDNGRSCPPCHEVCKGRCWGPGSEDCQTLTKTICAPQCNGHCFGPNPNQCCHDECAGGCSGPQDTDCFACRHFNDSGACVPRCPQPLVYNKLTFQLEPNPHTKYQYGGVCVASCPHNFVVDQTSCVRACPPDKMEVDKNGLKMCEPCGGLCPKACEGTGSGSRFQTVDSSNIDGFVNCTKILGNLDFLITGLNGDPWHKIPALDPEKLNVFRTVREITGYLNIQSWPPHMHNFSVFSNLTTIGGRSLYNRGFSLLIMKNLNVTSLGFRSLKEISAGRIYISANRQLCYHHSLNWTKVLRGPTEERLDIKHNRPRRDCVAEGKVCDPLCSSGGCWGPGPGQCLSCRNYSRGGVCVTHCNFLNGEPREFAHEAECFSCHPECQPMEGTATCNGSGSDTCAQCAHFRDGPHCVSSCPHGVLGAKGPIYKYPDVQNECRPCHENCTQGCKGPELQDCLGQTLVLIGKTHLTMALTVIAGLVVIFMMLGGTFLYWRGRRIQNKRAMRRYLERGESIEPLDPSEKANKVLARIFKETELRKLKVLGSGVFGTVHKGVWIPEGESIKIPVCIKVIEDKSGRQSFQAVTDHMLAIGSLDHAHIVRLLGLCPGSSLQLVTQYLPLGSLLDHVRQHRGALGPQLLLNWGVQIAKGMYYLEEHGMVHRNLAARNVLLKSPSQVQVADFGVADLLPPDDKQLLYSEAKTPIKWMALESIHFGKYTHQSDVWSYGVTVWELMTFGAEPYAGLRLAEVPDLLEKGERLAQPQICTIDVYMVMVKCWMIDENIRPTFKELANEFTRMARDPPRYLVIKRESGPGIAPGPEPHGLTNKKL

In [49]:
# random record
#n = Int(floor(length(oncodb)*rand()))
#println(n)

refseq_id = oncodb[n]["gene"]["curatedRefSeq"]
seq = fetch_sequence(refseq_id)
seq = revcomp(seq)
println("SVN at " * oncodb[n]["proteinStart"] * " in given protein")
SVN
offset = 3 * oncodb[n]["proteinStart"]
refseq_codon = seq[offset:offset + 2]

println("Codon found is " * refseq_codon * " which encodes Amino acid " * codon_aa[refseq_codon])
println("Reference residue reported is " * oncodb[n]["refResidues"])

for i in -3:3
    foo = offset + i
    bar = foo + 2
    cdn = seq[foo:bar]
    print(i) 
    print("\t" *cdn * "\t")
    println(codon_aa[cdn] ) 
end

1359
Codon found is CTG which encodes Amino acid L
Reference residue reported is R
-3	GTA	V
-2	TAC	Y
-1	ACT	T
0	CTG	L
1	TGT	C
2	GTA	V
3	TAT	Y


In [39]:
for vc in variant_choices 
    println(vc, "\t", SNV(vc))
end
# 0 based index here

Replacement("CGG","CAG",0.146)	Varloc(1,"G>A")
Replacement("CGA","CAA",0.0297)	Varloc(1,"G>A")


In [40]:
# constructor
possible_variants = Array{Varloc,1}();                          

In [41]:
println("Reference codon observed \t",refseq_codon)
println(possible_variants)

Reference codon observed 	CTG
Varloc[]


In [42]:
refseq_id * ":c." * string(offset - possible_variants[1].index) * possible_variants[1].transition

LoadError: BoundsError: attempt to access 0-element Array{Varloc,1} at index [1]

Striking out finding variants with this type of syntax "NM_138621.4:c.459C>T"  
try to get them all and see how it goes 

In [43]:
for rec in oncodb
    refseq_id  = rec["gene"]["curatedRefSeq"]
    seq = fetch_sequence(refseq_id)
    offset = 3 * rec["proteinStart"]
    refseq_codon = seq[offset:offset+2]
    key = rec["refResidues"] * rec["variantResidues"]
    variant_choices = lookup[key]
    rec["possible_allele"]=[]
    
    for vc in variant_choices
        if refseq_codon == vc.ref_codon
            codon_snv = SNV(vc)   
            push!(rec["possible_allele_id"], 
                refseq_id * ":c." * string(offset - codon_snv.index) * codon_snv.transition)
        end    
    end

    #if 1 > length(rec["possible_allele_id"])
    #    println("corblimy, Observed ", refseq_codon ," at ", offset,
    #        " BP within ", refseq_id, " but it is not found in\n", variant_choices)
    #end   
end

LoadError: MethodError: no method matching *(::Void, ::Void)[0m
Closest candidates are:
  *(::Any, ::Any, [1m[31m::Any[0m, [1m[31m::Any...[0m) at operators.jl:138[0m

In [44]:
code_llvm(is_snv, (String,String))


define i8 @julia_is_snv_71779(%jl_value_t*, %jl_value_t*) #0 {
top:
  %2 = call i32 @jlsys_getindex_44220(%jl_value_t* %0, i64 1)
  %3 = call i32 @jlsys_getindex_44220(%jl_value_t* %1, i64 1)
  %4 = icmp eq i32 %2, %3
  br i1 %4, label %if6, label %L3

L3:                                               ; preds = %top, %if6
  %"#temp#1.0" = phi i64 [ 0, %if6 ], [ 1, %top ]
  %5 = call i32 @jlsys_getindex_44220(%jl_value_t* %0, i64 2)
  %6 = call i32 @jlsys_getindex_44220(%jl_value_t* %1, i64 2)
  %7 = icmp eq i32 %5, %6
  br i1 %7, label %if6.1, label %L3.1

if6:                                              ; preds = %top
  br label %L3

if6.1:                                            ; preds = %L3
  br label %L3.1

L3.1:                                             ; preds = %if6.1, %L3
  %"#temp#1.0.1" = phi i64 [ 0, %if6.1 ], [ 1, %L3 ]
  %8 = call i32 @jlsys_getindex_44220(%jl_value_t* %0, i64 3)
  %9 = call i32 @jlsys_getindex_44220(%jl_value_t* %1, i64 3)
  %10 = icmp eq i32 %8, 

In [45]:
oncodb[6]["possible_allele_id"]


LoadError: KeyError: key "possible_allele_id" not found

Just for fun look at how to go from go from high levle python-y script-y  
all the way down to assembly with one of the little functions defined above

In [46]:
code_native(is_snv, (String,String))

	.text
Filename: In[5]
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%r15
	pushq	%r14
	pushq	%r13
	pushq	%r12
	pushq	%rbx
	pushq	%rax
	movq	%rsi, %r14
	movq	%rdi, %r15
Source line: 5
	movabsq	$getindex, %r13
	movl	$1, %r12d
	movl	$1, %esi
	callq	*%r13
	movl	%eax, %ebx
	movl	$1, %esi
	movq	%r14, %rdi
	callq	*%r13
	cmpl	%eax, %ebx
	jne	L64
	xorl	%r12d, %r12d
L64:
	movq	%r12, -48(%rbp)
	movl	$2, %esi
	movq	%r15, %rdi
	callq	*%r13
	movl	%eax, %ebx
	movl	$2, %esi
	movq	%r14, %rdi
	callq	*%r13
	movl	$1, %r12d
	cmpl	%eax, %ebx
	jne	L105
	xorl	%r12d, %r12d
L105:
	movl	$3, %esi
	movq	%r15, %rdi
	callq	*%r13
	movl	%eax, %ebx
	movl	$3, %esi
	movq	%r14, %rdi
	callq	*%r13
	movl	$1, %ecx
	cmpl	%eax, %ebx
	jne	L140
	xorl	%ecx, %ecx
L140:
	addq	-48(%rbp), %r12
	addq	%rcx, %r12
Source line: 7
	cmpq	$1, %r12
	sete	%al
	addq	$8, %rsp
	popq	%rbx
	popq	%r12
	popq	%r13
	popq	%r14
	popq	%r15
	popq	%rbp
	retq
	nopl	(%rax)


M.Dayhoff's PAM1 
http://www.deduveinstitute.be/~opperd/private/pam1.html